<a href="https://colab.research.google.com/github/Trevorchenmsu/Road-Challenge-Event-Detection-for-Situation-Awareness-in-Autonomous-Driving/blob/main/V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ********** step 1: set up google colab, enter home folder **********
from google.colab import drive
drive.mount('/content/drive/')
import os
home = "/content/drive/MyDrive/RoadEvents"
os.chdir(home)
os.getcwd()

Mounted at /content/drive/


'/content/drive/MyDrive/RoadEvents'

In [3]:
# ********** step 2: download road dataset **********
os.chdir(home)
!mkdir road
os.chdir(os.path.join(home, "road"))
os.getcwd()

# option 1: visit https://github.com/gurkirt/road-dataset/tree/master/road, download get_dataset.sh and store it in road folder
!bash get_dataset.sh

# option 2: run the commands below
#Downloading the videos zip
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YQ9ap3o9pqbD0Pyei68rlaMDcRpUn-qz' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1YQ9ap3o9pqbD0Pyei68rlaMDcRpUn-qz" -O videos.zip
#Unzipping the videos
unzip videos.zip
#Removing the zip file
rm videos.zip
#Downloading annotation file for training and validation in json format
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1HAJpdS76TVK56Qvq1jXr-5hfFCXKHRZo' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1HAJpdS76TVK56Qvq1jXr-5hfFCXKHRZo" -O road_trainval_v1.0.json 
#Downloading the instance counts file in json format
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1NfSoI1yVTA46YY7AwVIGRolAqtWfoa8V' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1NfSoI1yVTA46YY7AwVIGRolAqtWfoa8V" -O instance_counts.json 

 # install ffmpeg for extracting image from video
!sudo apt install ffmpeg

# visit https://github.com/gurkirt/road-dataset/blob/master/extract_videos2jpgs.py and download it to home folder
os.chdir(home)
os.chdir(os.path.join(home, "road"))
os.getcwd()

# extract rbg-img from video and store them in road folder
!python extract_videos2jpgs.py /content/drive/MyDrive/RoadEvents/road/

'/content/drive/MyDrive/RoadEvents/road'

In [5]:
# ********** step 3: downloads pre-trained weights **********
os.chdir(home)
!mkdir kinetics-pt
os.chdir(os.path.join(home, "kinetics-pt"))
os.getcwd()

# option 1: visit https://github.com/gurkirt/3D-RetinaNet/blob/master/kinetics-pt/get_kinetics_weights.sh, store it in kinetics-pt folder
!bash get_kinetics_weights.sh

# optiono 2: run the codes below
#Downloading Kinetics weights
wget --load-cookies cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=17xiC_Wrdv1noD9NZmgXGIZQWSnW0wnxP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=17xiC_Wrdv1noD9NZmgXGIZQWSnW0wnxP" -O resnet50C2D.pth
rm cookies.txt
wget --load-cookies cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XBMs4TLt2H378M_a0k23l8let-Ae2AlB' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1XBMs4TLt2H378M_a0k23l8let-Ae2AlB" -O resnet50I3D-NL.pth
rm cookies.txt
wget --load-cookies cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ZpbvJzvnDxJmKCFTs9wKmmA2qvm2aFBX' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1ZpbvJzvnDxJmKCFTs9wKmmA2qvm2aFBX" -O resnet50I3D.pth
rm cookies.txt

'/content/drive/MyDrive/RoadEvents/kinetics-pt'

In [2]:
# ********** step 4: install dependencies: Pytorch, torchvision, tensorboardX **********
!pip install torch==1.9.0 torchvision==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install tensorboardx

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 124 kB 8.6 MB/s 


In [ ]:
# ********** step 5: download baseline model: 3D-RetinaNet **********
os.chdir(home)
os.getcwd()
!git clone https://github.com/gurkirt/3D-RetinaNet.git

In [4]:
# ********** step 6: testing and building tubes with pre-trained model **********
os.chdir(os.path.join(home, "3D-RetinaNet"))
os.getcwd()

# trick: 
# (1) visit https://drive.google.com/drive/folders/1yrzf1VKscOlysXxKDh4N_6uRvt8-dxa- to 
#     download pretrained model: model_000030_resnet50I3D.pth; 
# (2) run this block, and it will show error: cannot find model: model_000030.pth
# (3) rename the downloaded model_000030_resnet50I3D.pth as model_000030.pth and 
#     store it at:  /content/drive/MyDrive/RoadEvents/road/cache/resnet50I3D512-Pkinetics-b4s8x1x1-roadt3-h3x3x3/

!python main.py \
  /content/drive/MyDrive/RoadEvents/ \
  /content/drive/MyDrive/RoadEvents/ \
  /content/drive/MyDrive/RoadEvents/kinetics-pt/ \
  --MODE=gen_dets --MODEL_TYPE=I3D --TEST_SEQ_LEN=8 \
  --TRAIN_SUBSETS=train_3 --SEQ_LEN=8 --BATCH_SIZE=4 --LR=0.0041 

Streaming output truncated to the last 5000 lines.
  torch.index_select(x2, 0, idx, out=xx2)
/content/drive/MyDrive/RoadEvents/3D-RetinaNet/modules/box_utils.py:371: UserWarning: An output with one or more elements was resized since it had shape [518], which does not match the required output shape [419].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/aten/src/ATen/native/Resize.cpp:23.)
  torch.index_select(y2, 0, idx, out=yy2)
/content/drive/MyDrive/RoadEvents/3D-RetinaNet/modules/box_utils.py:368: UserWarning: An output with one or more elements was resized since it had shape [419], which does not match the required output shape [315].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reus

In [3]:
# ********** step 7: training **********
os.chdir(os.path.join(home, "3D-RetinaNet"))
os.getcwd()
!python main.py \
  /content/drive/MyDrive/RoadEvents/ \
  /content/drive/MyDrive/RoadEvents/ \
  /content/drive/MyDrive/RoadEvents/kinetics-pt/ \
  --MODE=train --ARCH=resnet50 --MODEL_TYPE=I3D --DATASET=road --TRAIN_SUBSETS=train_3 \
  --SEQ_LEN=8 --TEST_SEQ_LEN=8 --BATCH_SIZE=4 --LR=0.0041

Your working directories are::
LOAD::>  /content/drive/MyDrive/RoadEvents/ 
SAVE::>  /content/drive/MyDrive/RoadEvents/
Your model will be initialized using /content/drive/MyDrive/RoadEvents/kinetics-pt/resnet50I3D.pth
[INFO: main.py:  190]: 3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
[INFO: datasets.py:  370]: Number of agent: all :: 11 to use: 10
[INFO: datasets.py:  370]: Number of action: all :: 22 to use: 19
[INFO: datasets.py:  370]: Number of loc: all :: 12 to use: 12
[INFO: datasets.py:  370]: Number of duplex: all :: 152 to use: 39
[INFO: datasets.py:  370]: Number of triplet: all :: 1620 to use: 68
[INFO: datasets.py:  452]: Frames with Boxes are 5307 out of 6000 in 2014-06-26-09-53-12_stereo_centre_02
[INFO: datasets.py:  459]: number of start frames: 750
[INFO: datasets.py:  452]: Frames with Boxes are 5597 out of 6001 in 2014-07-14-14-49-50_stereo_centre_01
[INFO: datasets.py:  459]: number of start frames: 750
[INFO: datasets.py:  452]: Frames with Boxes are 5345